## COVID Pipeline Deployment on UCS

### Clone Cisco Kubeflow Starter Pack Repository

In [3]:
! git clone https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 1681, done.
remote: Total 1681 (delta 0), reused 0 (delta 0), pack-reused 1681
Receiving objects: 100% (1681/1681), 4.45 MiB | 6.38 MiB/s, done.
Resolving deltas: 100% (661/661), done.


### Import libraries

In [21]:
import kfp
import os
import calendar
import time
from kubernetes import client


### Component files Declarations

In [22]:
path='cisco-kubeflow-starter-pack/apps/healthcare/covid-forecasting/onprem/pipelines/'
component_root_preprocess = path + 'components/covid_preprocess/'
component_root_train = path + 'components/covid_train/'
component_root_serve = path + 'components/covid_serve/'

#### Components Description
covid_datadownload_op &nbsp; - &nbsp; Downloads dataset from GCP bucket and stores in nfs-volume. <br>
covid_preprocess_op &nbsp; - &nbsp; Preprocess the raw dataset to make it suitable for training. <br>
covid_train_op &nbsp; - &nbsp; Load dataset from nfs-volume, train COVID-19 model and saves model in nfs-volume. <br>
covid_serve_op &nbsp; - &nbsp; Load COVID-19 model from nfs-volume and serve the model using tf-serving for later prediction. <br> 
covid_upload_result_op &nbsp; - &nbsp; Uploading the prediction results into GCS bucket for the sake of visualisation. <br> 



### Load components from respective .YAML config files

In [23]:
covid_preprocess_op = kfp.components.load_component_from_file(os.path.join(component_root_preprocess, 'component.yaml'))
covid_train_op = kfp.components.load_component_from_file(os.path.join(component_root_train, 'component.yaml'))
covid_serve_op = kfp.components.load_component_from_file(os.path.join(component_root_serve, 'component.yaml'))

### Define Volume and Volume Mounts

In [24]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

### Create Timestamp

In [25]:
timestamp = str(calendar.timegm(time.gmtime()))
print(timestamp)

1588242007



### Define pipeline function

In [26]:
def covid_pipeline():
    
    
    #Define Task for Preprocess covid data
    covid_preprocess_task = covid_preprocess_op()
    covid_preprocess_task.add_volume(nfs_volume)
    covid_preprocess_task.add_volume_mount(nfs_volume_mount)
    
    #Define Task for Training model for COVID-19 prediction
    covid_train_task = covid_train_op(epochs=10)
    covid_train_task.add_volume(nfs_volume)
    covid_train_task.add_volume_mount(nfs_volume_mount)
    covid_train_task.after(covid_preprocess_task)
    
    #Define Task for Serving for COVID-19 prediction
    covid_serve_task = covid_serve_op(timestamp=timestamp)
    covid_serve_task.add_volume(nfs_volume)
    covid_serve_task.add_volume_mount(nfs_volume_mount)
    covid_serve_task.after(covid_train_task)    

### Create & Execute a COVID Forcast Pipeline

In [27]:
#Create a pipeline run
kfp.Client().create_run_from_pipeline_func(covid_pipeline, arguments={})

RunPipelineResult(run_id=361ddb67-faf7-416e-9212-874bc4d87009)

### Retrieve ClusterIP of Serving Service & Predict using curl request

In [25]:
%%bash
serve_ip=$(kubectl -n kubeflow get svc | grep covid-service | awk '{print $3}')
echo $serve_ip
curl -v http://$serve_ip:8500/v1/models/Model_Covid:predict -d '{"signature_name":"serving_default","instances":[{"input1":[[1.0986122886681098, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.6931471805599453],[0.0, 0.0],[1.0986122886681098, 0.0],[0.6931471805599453, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0]], "input2":[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]}, {"input1":[[1.0986122886681098, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.6931471805599453],[0.0, 0.0],[1.0986122886681098, 0.0],[0.6931471805599453, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0],[0.0, 0.0]], "input2":[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]} ]}'

10.99.104.192
{
    "predictions": [[3.71306396e-06, 6.48195453e-07, 3.59812134e-06, 1.01000251e-06, 6.99493876e-06, 1.83913096e-06, 4.58340446e-06, 9.02905526e-07, 9.59076442e-06, 1.22532163e-06, 7.27896304e-06, 1.20176969e-06, 5.31440946e-06, 1.50903679e-06, 2.09175e-06, 7.40330222e-07, 1.14892537e-05, 1.35839412e-06, 1.05419758e-05, 5.92884078e-07, 1.05328818e-05, 1.12036582e-06, 2.78480093e-05, 1.24785288e-06, 2.02848933e-05, 6.77313153e-07, 2.69135126e-05, 1.8475198e-06, 3.51139279e-05, 1.95976213e-06, 2.96442777e-05, 5.46645367e-07, 8.02301511e-05, 1.52324856e-06, 0.000855864957, 8.5055035e-07, 0.085144192, 9.31548186e-07, 0.0946462601, 1.05992603e-06, 0.138091743, 1.53388135e-06, 0.191763729, 1.13644512e-06, 0.177047938, 1.91902927e-06, 0.312126338, 8.4027e-07], [3.71306396e-06, 6.48195453e-07, 3.59812134e-06, 1.01000251e-06, 6.99493876e-06, 1.83913096e-06, 4.58340446e-06, 9.02905526e-07, 9.59076442e-06, 1.22532163e-06, 7.27896304e-06, 1.20176969e-06, 5.31440946e-06, 1.50903679e

*   Trying 10.99.104.192...
* TCP_NODELAY set
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0* Connected to 10.99.104.192 (10.99.104.192) port 8500 (#0)
> POST /v1/models/Model_Covid:predict HTTP/1.1
> Host: 10.99.104.192:8500
> User-Agent: curl/7.58.0
> Accept: */*
> Content-Length: 3908
> Content-Type: application/x-www-form-urlencoded
> Expect: 100-continue
> 
< HTTP/1.1 100 Continue
} [3908 bytes data]
* We are completely uploaded and fine
< HTTP/1.1 200 OK
< Content-Type: application/json
< Date: Thu, 30 Apr 2020 14:12:30 GMT
< Content-Length: 1516
< 
{ [1516 bytes data]
100  5424  100  1516  100  3908  14862  38313 --:--:-- --:--:-- --:--:-- 53176
* Connection #0 to host 10.99.104.192 left intact
